In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [3]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:02<00:00,  1.51it/s]


In [4]:
print(df_hist['purchase_date'].max())
print(df_hist['purchase_date'].mih())

2018-02-28 23:59:51
2017-01-01 00:00:08


In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [4]:
for df in [df_hist, df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['hour'] = df['purchase_date'].dt.hour
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [5]:
df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
df_new.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)

In [6]:
df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_new['yyyymm'] =  df_new['purchase_date'].map(lambda x: str(x)[:7])
df_hist['yyyymm_week'] = df_hist['yyyymm'] + df_hist['weekofyear'].map(lambda x: '-' + str(x))
df_new['yyyymm_week'] = df_new['yyyymm'] + df_new['weekofyear'].map(lambda x: '-' +  str(x))
df_hist.head()

,authorized_flag,card_id,month_lag,purchase_date,state_id,subsector_id,weekofyear,month,hour,month_diff,date_diff,yyyymm,yyyymm_week
0,Y,C_ID_4e6213e9bc,-8,2017-06-25 15:33:07,16,37,25,6,15,18,554,2017-06,2017-06-25
1,Y,C_ID_4e6213e9bc,-7,2017-07-15 12:10:45,16,16,28,7,12,17,534,2017-07,2017-07-28
2,Y,C_ID_4e6213e9bc,-6,2017-08-09 22:04:29,16,37,32,8,22,16,509,2017-08,2017-08-32
3,Y,C_ID_4e6213e9bc,-5,2017-09-02 10:06:26,16,34,35,9,10,16,486,2017-09,2017-09-35
4,Y,C_ID_4e6213e9bc,-11,2017-03-10 01:14:19,16,37,10,3,1,22,662,2017-03,2017-03-10


In [29]:

df_hist.head(10)

,card_id,purchase_date,date_diff,month_diff,yyyymm,yyyymm_week,row_no
0,C_ID_00007093c1,2017-02-14 14:00:43,685,22,2017-02,2017-02-7,1
1,C_ID_00007093c1,2017-02-14 15:47:45,685,22,2017-02,2017-02-7,2
2,C_ID_00007093c1,2017-02-16 15:37:58,683,22,2017-02,2017-02-7,3
3,C_ID_00007093c1,2017-02-20 12:19:01,679,22,2017-02,2017-02-8,4
4,C_ID_00007093c1,2017-03-03 00:24:15,669,22,2017-03,2017-03-9,5
5,C_ID_00007093c1,2017-03-06 18:50:42,665,22,2017-03,2017-03-10,6
6,C_ID_00007093c1,2017-03-08 15:05:29,663,22,2017-03,2017-03-10,7
7,C_ID_00007093c1,2017-03-08 16:12:35,663,22,2017-03,2017-03-10,8
8,C_ID_00007093c1,2017-03-11 15:04:31,660,22,2017-03,2017-03-10,9
9,C_ID_00007093c1,2017-03-11 15:10:15,660,22,2017-03,2017-03-10,10


### first 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する
### last 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する

In [30]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df = df_new
hist_new_name = 'new'
aggs = {}
aggs['date_diff'] = ['mean', 'std']

df = df[[key, 'purchase_date', 'date_diff', 'month_diff', 'yyyymm', 'yyyymm_week']]


for i in range(5, 31, 5):
    
    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df = utils.row_number(df=df, level=key)

    feat_n = f'{hist_new_name}_first'
    first = df.query(f"row_no<={i}")
    tmp = first.groupby(key)['date_diff'].agg(aggs)
    new_cols = get_new_columns(f'{feat_n}{i}', aggs)
    tmp.columns = new_cols
    df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
    df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
    
    
    
for i in range(5, 31, 5):
    
    df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
    df = utils.row_number(df=df, level=key)
    
    feat_n = f'{hist_new_name}_last'
    last = df.query(f"row_no<={i}")
    tmp = last.groupby(key)['date_diff'].agg(aggs)
    new_cols = get_new_columns(f'{feat_n}{i}', aggs)
    tmp.columns = new_cols
    df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
    df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
    
# sys.exit()

#========================================================================
# これはあまり意味がないな。期間featureのおかしいバージョン
#========================================================================
# for i in range(5, 31, 5):
#     df_train[f'{hist_new_name}_trans{i}_date_diff_mean'] = df_train[f'{hist_new_name}_last{i}_date_diff_mean'] - df_train[f'{hist_new_name}_first{i}_date_diff_mean']
#     df_test[f'{hist_new_name}_trans{i}_date_diff_mean'] = df_test[f'{hist_new_name}_last{i}_date_diff_mean'] - df_test[f'{hist_new_name}_first{i}_date_diff_mean']
#     df_train[f'{hist_new_name}_trans{i}_date_diff_std'] = df_train[f'{hist_new_name}_last{i}_date_diff_std'] - df_train[f'{hist_new_name}_first{i}_date_diff_std']
#     df_test[f'{hist_new_name}_trans{i}_date_diff_std'] = df_test[f'{hist_new_name}_last{i}_date_diff_std'] - df_test[f'{hist_new_name}_first{i}_date_diff_std']
    
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 62.77it/s]


,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_first5_date_diff_mean,new_first5_date_diff_std,new_first10_date_diff_mean,new_first10_date_diff_std,new_first15_date_diff_mean,new_first15_date_diff_std,new_first20_date_diff_mean,new_first20_date_diff_std,new_first25_date_diff_mean,new_first25_date_diff_std,new_first30_date_diff_mean,new_first30_date_diff_std,new_last5_date_diff_mean,new_last5_date_diff_std,new_last10_date_diff_mean,new_last10_date_diff_std,new_last15_date_diff_mean,new_last15_date_diff_std,new_last20_date_diff_mean,new_last20_date_diff_std,new_last25_date_diff_mean,new_last25_date_diff_std,new_last30_date_diff_mean,new_last30_date_diff_std,new_trans5_date_diff_mean,new_trans5_date_diff_std,new_trans10_date_diff_mean,new_trans10_date_diff_std,new_trans15_date_diff_mean,new_trans15_date_diff_std,new_trans20_date_diff_mean,new_trans20_date_diff_std,new_trans25_date_diff_mean,new_trans25_date_diff_std,new_trans30_date_diff_mean,new_trans30_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,297.6,3.049590,290.200000,9.077445,284.000000,11.789826,278.600000,14.106363,274.782609,16.602181,274.782609,16.602181,252.8,5.932959,259.800000,8.599742,264.933333,10.395512,271.050000,14.369101,274.782609,16.602181,274.782609,16.602181,-44.8,2.883369,-30.4,-4.777030e-01,-19.066667,-1.394314e+00,-7.55,2.627377e-01,0.0,1.421085e-14,0.0,1.421085e-14
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,313.0,24.238399,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,301.8,25.587106,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,307.000000,26.191602,-11.2,1.348707,0.0,-3.552714e-15,0.000000,-3.552714e-15,0.00,-3.552714e-15,0.0,-3.552714e-15,0.0,-3.552714e-15
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,247.0,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,247.0,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,247.000000,NaN,0.0,NaN,0.0,NaN,0.000000,NaN,0.00,NaN,0.0,NaN,0.0,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,276.0,14.966630,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,263.0,5.830952,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,270.857143,15.060197,-13.0,-9.135678,0.0,-1.776357e-14,0.000000,-1.776357e-14,0.00,-1.776357e-14,0.0,-1.776357e-14,0.0,-1.776357e-14
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,301.6,4.335897,296.700000,6.056218,291.133333,9.855141,286.600000,11.731200,282.360000,13.585777,278.633333,15.004559,251.6,4.277850,255.200000,4.984420,258.200000,6.026134,261.550000,7.996545,264.760000,9.773604,269.033333,13.202621,-50.0,-0.058047,-41.5,-1.071798e+00,-32.933333,-3.829008e+00,-25.05,-3.734655e+00,-17.6,-3.812173e+00,-9.6,-1.801938e+00


In [31]:
if hist_new_name=='hist':
    fname = '130_ddf'
elif hist_new_name=='new':
    fname = '131_ddf'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

### 132 date_diffの前半、後半における違いを特徴にする. 全体verも

In [34]:
df_hist['purchase_date'] =  df_hist['purchase_date'].map(lambda x: str(x)[:10])
df_new['purchase_date'] =  df_new['purchase_date'].map(lambda x: str(x)[:10])

In [46]:
from joblib import Parallel, delayed
from dateutil.parser import parse

# ========================================================================
# Time Aggregate Function
# ========================================================================
# def diff_of_days(day1, day2):
def diff_of_days(args):
    uid = args[0]
    day1 = args[1]
    day2 = args[2]
    if str(type(day1)).count('int') or str(type(day2)).count('int'):
        return [day1, day2]
    days = (parse(day1) - parse(day2)).days
    return [uid, days, day1]

# 前回購買日との差分
def diff_shift_date(df, lag):
    
    # shiftは正が下方向、負が上方向にズレる
    df = df[[key, 'purchase_date']].drop_duplicates()
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['shift1_date'] = df.groupby(key)['purchase_date'].shift(lag)
    df = df[[key, 'purchase_date', 'shift1_date']].dropna()
#     id_list = df[key].values
#     shift1 = df['shift1_date'].values
#     shift0 = df['purchase_date'].values
#     p_list = Parallel(n_jobs=-1)([delayed(diff_of_days)(args) for args in zip(id_list, shift0, shift1)])
    df['date_diff'] = (df['purchase_date'] - df['shift1_date']).dt.days
    return df
    
#     df_diff = pd.DataFrame(np.array(p_list), columns=[key, f'diff_days_lag{lag}@', 'purchase_date'])
#     return df_diff
    
# for i in tqdm(range(1, 4, 1)):
for i in tqdm(range(1, 2, 1)):
#     diff = diff_shift_date(df_hist, i)
    diff = diff_shift_date(df_new, i)
#     df_hist = df_hist.merge(diff, how='left', on=[key, 'purchase_date']) 
    
# for col in [col for col in df_hist.columns if col.count('@')]:
#     df_hist[col] = df_hist[col].astype('float')
    
# df_hist.head()
diff.head()


100%|██████████| 1/1 [00:02<00:00,  2.36s/it]


,card_id,purchase_date,shift1_date,date_diff
1,C_ID_00007093c1,2018-04-09,2018-04-03,6
3,C_ID_0001238066,2018-03-03,2018-03-01,2
4,C_ID_0001238066,2018-03-04,2018-03-03,1
5,C_ID_0001238066,2018-03-10,2018-03-04,6
6,C_ID_0001238066,2018-03-16,2018-03-10,6


In [47]:
# df = df_hist[[key, 'purchase_date', 'diff_days_lag1@']].drop_duplicates()
df = diff[[key, 'purchase_date', 'date_diff']].drop_duplicates()
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
hist_new_name = 'hist'
hist_new_name = 'new'
aggs = {}
aggs['date_diff'] = ['mean', 'std']

df.sort_values(by=[key, 'purchase_date'], inplace=True)
df = utils.row_number(df=df, level=key)
for i in range(5, 31, 5):
    
    feat_n = f'{hist_new_name}_first'
    first = df.query(f"row_no<={i}")
    tmp = first.groupby(key)['date_diff'].agg(aggs)
    new_cols = get_new_columns(f'{feat_n}{i}', aggs)
    tmp.columns = new_cols
    df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
    df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
    
    
df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
df = utils.row_number(df=df, level=key)
for i in range(5, 31, 5):
     
    feat_n = f'{hist_new_name}_last'
    last = df.query(f"row_no<={i}")
    tmp = last.groupby(key)['date_diff'].agg(aggs)
    new_cols = get_new_columns(f'{feat_n}{i}', aggs)
    tmp.columns = new_cols
    df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
    df_test = df_test.merge(tmp.reset_index(), how='left', on=key)


100%|██████████| 3/3 [00:00<00:00, 172.35it/s]

In [49]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_first5_date_diff_mean,new_first5_date_diff_std,new_first10_date_diff_mean,new_first10_date_diff_std,new_first15_date_diff_mean,new_first15_date_diff_std,new_first20_date_diff_mean,new_first20_date_diff_std,new_first25_date_diff_mean,new_first25_date_diff_std,new_first30_date_diff_mean,new_first30_date_diff_std,new_last5_date_diff_mean,new_last5_date_diff_std,new_last10_date_diff_mean,new_last10_date_diff_std,new_last15_date_diff_mean,new_last15_date_diff_std,new_last20_date_diff_mean,new_last20_date_diff_std,new_last25_date_diff_mean,new_last25_date_diff_std,new_last30_date_diff_mean,new_last30_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,2.0,1.00000,2.5,2.460804,2.6,2.292846,2.75,2.359193,2.75,2.359193,2.750000,2.359193,3.2,2.774887,3.0,2.357023,3.000000,2.645751,2.75,2.359193,2.75,2.359193,2.750000,2.359193
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,19.0,21.00000,19.0,21.000000,19.0,21.000000,19.00,21.000000,19.00,21.000000,19.000000,21.000000,19.0,21.000000,19.0,21.000000,19.000000,21.000000,19.00,21.000000,19.00,21.000000,19.000000,21.000000
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,8.2,5.80517,7.0,5.966574,7.0,5.966574,7.00,5.966574,7.00,5.966574,7.000000,5.966574,5.0,3.807887,7.0,5.966574,7.000000,5.966574,7.00,5.966574,7.00,5.966574,7.000000,5.966574
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,2.6,3.04959,2.8,2.250926,2.4,2.028370,2.15,1.814416,2.24,1.738774,2.192308,1.720912,2.4,1.673320,2.0,1.247219,1.866667,1.245946,2.10,1.372665,2.20,1.755942,2.192308,1.720912


In [48]:
if hist_new_name=='hist':
    fname = '132_ddf'
elif hist_new_name=='new':
    fname = '133_ddf'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('diff_days')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

In [54]:
df = diff[[key, 'purchase_date', 'date_diff']].drop_duplicates()
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
hist_new_name = 'new'
aggs = {}
aggs['date_diff'] = ['mean', 'std']

df.sort_values(by=[key, 'purchase_date'], inplace=True)
df = utils.row_number(df=df, level=key)
tmp = df.groupby(key)['date_diff'].agg(aggs)
new_cols = get_new_columns(f'{hist_new_name}', aggs)
tmp.columns = new_cols
df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
df_train.head()


100%|██████████| 3/3 [00:00<00:00, 160.55it/s]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_date_diff_mean,new_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,2.750000,2.359193
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,19.000000,21.000000
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,7.000000,5.966574
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,2.192308,1.720912
